https://github.com/marcrobledo/savegame-editors/issues/291#issuecomment-1575410533 claims that footprint.sav contains Hero's Path data.

In [2]:
from google.colab import drive
from pathlib import Path

DRIVE_ROOT = Path('drive')
SAVES_DIR = DRIVE_ROOT / "MyDrive" / "TOTK Hero's Path" / "saves"

drive.mount(str(DRIVE_ROOT))

Mounted at drive


In [5]:
from functools import cached_property
from pathlib import Path

class Save:
  _root: Path

  def __init__(self, savedir: Path):
    self._root = savedir

  @cached_property
  def footprint_raw(self):
    with open(self._root / 'footprint.sav', 'rb') as f:
      return f.read()



In [6]:
SAVES = {
    p.name: Save(p)
    for p in SAVES_DIR.iterdir()
    if p.is_dir()
}
SAVES

{'slot_00': <__main__.Save at 0x7b5039dab940>,
 'slot_01': <__main__.Save at 0x7b5039dabbb0>,
 'slot_02': <__main__.Save at 0x7b503ac410c0>,
 'slot_03': <__main__.Save at 0x7b5039dfc280>,
 'slot_04': <__main__.Save at 0x7b5039dfc550>,
 'slot_05': <__main__.Save at 0x7b5039dfcac0>}

## Basic diffing

As a start, try to take a binary diff of two save slots' footprint files. Tools like `xdelta3` or `bsdiff` can do this efficiently for binary files, but `difflib` in the Python standard library can do something similar.

In [ ]:
import difflib

matcher = difflib.SequenceMatcher(
    a=SAVES['slot_00'].footprint_raw,
    b=SAVES['slot_01'].footprint_raw
)
matcher.get_opcodes()

[('equal', 0, 76, 0, 76),
 ('replace', 76, 77, 76, 77),
 ('equal', 77, 272796, 77, 272796),
 ('replace', 272796, 614760, 272796, 614760)]

That takes a while to compute, but is somewhat informative.

These two slots are mostly the same in the first 256k (well, 256k + 10652), except the bytes at offset 76 and 77. Everything after 272796 is different, or at least not correlated in an interesting way.

Since it's not clear how these slots correlate to each other (Is one newer than the other? By how much?), do another pair as well.

In [ ]:
difflib.SequenceMatcher(
    a=SAVES['slot_01'].footprint_raw,
    b=SAVES['slot_02'].footprint_raw
).get_opcodes()

[('equal', 0, 76, 0, 76),
 ('replace', 76, 77, 76, 77),
 ('equal', 77, 272868, 77, 272868),
 ('replace', 272868, 614760, 272868, 614760)]

Looks like slot 2 has slightly more data in it than slot 1, since the `equal` segments in this second diff are $272868 - 272796 = 72$ bytes larger.

This looks like maybe the final `replace` in each of these is somewhat misleading, so take a closer look at the actual data. First, the 77ish bytes at the front.

In [ ]:
for name, s in SAVES.items():
  print(name, s.footprint_raw[:80].hex(sep=' '))

slot_00 04 03 02 01 f4 e0 47 00 58 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 59 68 a0 c5 01 00 00 00 6b 6f dc 37 00 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00 02 00 00 00 43 c2 e8 08 0c 0a 01 00
slot_01 04 03 02 01 f4 e0 47 00 58 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 59 68 a0 c5 01 00 00 00 6b 6f dc 37 00 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00 02 00 00 00 43 c2 e8 08 1e 0a 01 00
slot_02 04 03 02 01 f4 e0 47 00 58 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 59 68 a0 c5 01 00 00 00 6b 6f dc 37 00 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00 02 00 00 00 43 c2 e8 08 69 0a 01 00
slot_03 04 03 02 01 f4 e0 47 00 58 01 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 59 68 a0 c5 01 00 00 00 6b 6f dc 37 00 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00 02 00 00 00 43 c2 e8 08 c7 0a 01 00
slot_04 

It looks like the bytes at 76 and 77 are part of a count, since these look like they might be incrementing a value. The first byte doesn't obviously seem meaningful, but we see one that might be 0x09e5 when considered as a little-endian integer, and cycling through the other slots (starting from slot 4) and looping around gives a sequence that is strictly increasing.

Assuming that the slots are updated in sequence in the same way and the path log is simply appended to, this looks like it could be a count of points or data offset indicating how many points are stored in this log.

0. 0x09e5
1. 0x0a90
2. 0x0a0c
3. 0x0a1e
4. 0x0a69
5. 0x0ac7

Okay, so they're not completely in order. Perhaps the last slot is treated specially? Since the others do have increasing order going in sequence from slot $4, 0, 1, 2, 3$, perhaps slot 5 is reserved for manual saves; I believe the data here has a manual save as the most recent one, and slot 5 has the largest number.

After I first looked at the data, I guessed that it probably uses more than 16 bits for size so dumped up to byte 80 as a nice round number (20 32-bit words). I'm guessing this is actually a 32-bit number at file offsets 76..80, which would make the value for slot 5 0x00010a90.

Going further into trying to make sense of the numbers, we guessed that slot 1 has data up to offset 272868 and slot 0 only up to 272796; that's a difference of 72, or 0x48. If we treat the 32 bits at 76..80 as the hypothesized integers, do they have a similar difference?

In [ ]:
(
    int.from_bytes(SAVES['slot_01'].footprint_raw[76:80], byteorder='little')
    - int.from_bytes(SAVES['slot_00'].footprint_raw[76:80], byteorder='little')
)

18

18 times 4 is 72, so it seems like that could be an element count where each element in the log is 4 bytes in size.

Kevin Jensen [documented the Hero's Path log format for Breath of the Wild](https://gist.github.com/zephenryus/e46c797ccecf6134d4245bb9f2e5e2a5) which could be informative. The headers don't seem relevant here because TOTK doesn't split the track into multiple files and the header seems like it might be larger here, but BOTW uses 32-bit entries for each point in the track so it seems like that we've got the right idea here that slot 1 has 18 more 32-bit entries in it than slot 0 did.

So what does that look like? I'll take 16 bytes prior to the hypothesized end of slot 0's data and go to the end of slot 1's data, comparing their contents.

In [ ]:
for slot in ('slot_00', 'slot_01'):
  print(slot, SAVES[slot].footprint_raw[272796 - 16:272868].hex(sep=' '))

slot_00 b0 39 78 25 30 3d 98 25 30 3f 78 25 f0 3e 48 25 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
slot_01 b0 39 78 25 30 3d 98 25 30 3f 78 25 f0 3e 48 25 30 41 48 27 70 41 58 27 30 44 28 27 f0 44 48 27 70 44 08 27 f0 44 48 27 70 3f 88 26 30 38 d8 26 70 35 d8 26 b0 35 e8 26 f0 34 38 27 30 3c 28 27 b0 3e c8 25 30 45 58 27 70 47 b8 26 70 45 48 27 f0 43 f8 26 b0 3e 38 28


We already knew they're the same up to offset 272796, but here we can also see that the earlier slot is all zeroes past the point where its data seems to end.

As for what that counter at file offset 76 actually means, 0x10a0c (its value in slot 0) is 68108. Multiplying that by 4 bytes per entry is 272432 bytes of track data, and subtracting that from the file offset of slot 0's data endpoint indicates the data probably has a $272796 - 272432 = 364$ byte header: point data begins at file offset 364.

---

Going back to the points, I notice an interesting pattern in a couple points in slot 1, especially these bytes (aligned to 4 bytes and separated at the same boundaries):

```
f0 44 48 27: 0x274844f0
70 44 08 27: 0x27084470
f0 44 48 27: 0x274844f0
70 3f 88 26: 0x26883f70
```

The `f0 44` sequence appears twice; probably two points in the track that record the same location and state! The other adjacent points are probably very close in state and location, or possibly each entry is actually 8 bytes in size (and the counter records the size in multiples of 32 bits but always increments by more than that size).

The second line also differs from the first and third by only a few bits; it cleared bits 7 (0xf0 -> 0x70) and 22 (0x48 -> 0x08), assuming these are again little-endian integers. The documentation on BOTW's format says they're big-endian, so TOTK might also treat these as big-endian values but regardless of interpretation the difference between these two values is probably small.

## Cross-referencing with game output

Taking a break from just looking at data, let's see what the game says for each of these saves.

![Load screen screenshot](https://drive.google.com/uc?export=view&id=10NBHw1eeTjCkrofXjPziT80AVKznWCRF)

Recalling that `slot_04` appears to be the oldest and `slot_03` the newest autosave (assuming that `slot_05` is the manual save or something else), then `slot_00` as the second-oldest slot is the save dated 20:32 and `slot_01` is dated 20:34.

![20:32 map screen](https://drive.google.com/uc?export=view&id=10MdVzbrA1dxMo5NSPviXzoW1PDoeFZWj)

At 20:32 (slot 0), the displayed map coordinates are -249, 601, -1194.

![20:34 map screen](https://drive.google.com/uc?export=view&id=10KfBa6jEBKlEgr1_q_ZBcPtt3ik5-uAx)

In slot 1 (20:34), coordinates are -248, 648, -1225. The Hero's Path track shows a lot of recent movement in a fairly small area:

![20:34 map screen with Hero's Path](https://drive.google.com/uc?export=view&id=10QFqTJzvy59B4sL-QqM79gmrey385HIE)

Looking closely at the Hero's Path, the most recent point for each save may not be exactly the same as the player position on loading the save- clearly the player's current position is recorded elsewhere in the save data, which makes sense since it also needs to contain things like the player's current orientation.

We can also guess that Hero's Path doesn't store full X,Y,Z coordinates at each point because that would probably require more than 32 bits. The game doesn't actually show the player's Z coordinate (elevation) at each point, instead only indicating which map layer (Depths, Surface, Sky) the player was on by displaying parts of the track from different layers with dimmer lines. This probably means that it only records the current layer with a few bits, rather than a full Z coordinate in ~12 bits (assuming TOTK uses similar resolution to the 12 bits claimed for BOTW's X and Y coordinates).

Given some point data and known coordinates now, is there anything that looks like the coordinates in the data?

Starting with slot 0, our coordinates are -249, 601 and we're in the depths. The last 32-bit entry in that save's footprint file is `f0 3e 48 25`. To pattern-match it'll be easier to look at these in binary. I'll assume world coordinates are expressed as 12-bit numbers like they were in Breath of the Wild, considering for now just the absolute value of each coordinate (since BOTW is said to have used a sign-and-magnitude representation rater than two's complement):

* 249: `000011111001`
* 601: `001001011001`

The 32-bit word, as either little or big-endian:

* 0xf03e4825 (big-endian): `11110000001111100100100000100101`
* 0x25483ef0 (little-endian): `00100101010010000011111011110000`

It looks like the X coordinate appears in the big-endian version:

```
11110000001111100100100000100101 BE32
      000011111001               X
                                 Y?
```

but there's no obvious match to the Y coordinate here. This might just be because the last footprint point in this save isn't very near to the current player position.

```
00100101010010000011111011110000 LE32
              000011111001       X (249)
001001011001                     Y (601)
```

The little-endian interpretation looks like it's actually a better fit. If the player position is only moved slightly from the last footprint we'd expect the higher-order bits to mostly stay the same; with this placement the last footprint would be only a few units off from the player position: $(251, 596) - (249, 601) = (2, -5)$.

---

Using slot 1 to try to validate these assumptions, we have the coordinates (-248, 648) and bytes `b0 3e 38 28`.

```
00101000001110000011111010110000 LE32
              000011111000       X (248)
001010001000                     Y (648)
```

This is again a pretty small difference between current position and last footprint; a shift of $(2, 5)$.

# More samples

With a pretty good guess for the basic coordinates, we need more saves to look at the coordinates.

In [ ]:
from dataclasses import dataclass
from enum import Enum, auto

class Layer(Enum):
  SURFACE = auto()
  SKY = auto()
  DEPTHS = auto()

@dataclass(frozen=True)
class Footprints:
  points: tuple[int]
  known_x: int
  known_y: int
  known_layer: Layer

  def __len__(self):
    return len(self.points)

  @classmethod
  def from_sav(cls, path: Path, *args, **kwargs):
    with open(path, 'rb') as f:
      f.seek(76)
      count = int.from_bytes(f.read(4), byteorder='little')

      f.seek(364)
      points = []
      for _ in range(count):
        points.append(int.from_bytes(f.read(4), byteorder='little'))

    return cls(tuple(points), *args, **kwargs)

Here are a pair of saves where I warped around to a couple points, changing the sign of some coordinates and remaining on the surface.

1. Started at (-155, 1155)
2. Warped to (222, 1085)
3. Warped to (4632, -3712)

In [ ]:
MORE_SAVES_DIR = SAVES_DIR.parent

MORE_SAVES = [
    Footprints.from_sav(MORE_SAVES_DIR / f'{name}.sav', x, y, layer)
    for (name, (x, y, layer)) in {
        '001': (-155, 1155, Layer.SURFACE),
        '002': (4632, -3712, Layer.SURFACE),
    }.items()
]

print(len(MORE_SAVES[0]))
print(len(MORE_SAVES[1]))

51827
51835


In [ ]:
def dump_last_points(fp, count):
  for p in fp.points[-count:]:
    print(f'{p:#010x} {p:032b}')

dump_last_points(MORE_SAVES[0], 4)
print()
dump_last_points(MORE_SAVES[1], len(MORE_SAVES[1]) - len(MORE_SAVES[0]))

0x49482fe8 01001001010010000010111111101000
0x49082ca8 01001001000010000010110010101000
0x48e82ce8 01001000111010000010110011101000
0x48682968 01001000011010000010100101101000

0x483826ec 01001000001110000010011011101100
0x43d83788 01000011110110000011011110001000
0x43d8378c 01000011110110000011011110001100
0xe8048608 11101000000001001000011000001000
0xe8048608 11101000000001001000011000001000
0xe8048608 11101000000001001000011000001000
0xe8048608 11101000000001001000011000001000
0xe8048608 11101000000001001000011000001000


```
010010000110 10 000010100101 101000 0x48682968
        1158             165
010010000011 10 000010011011 101100 0x483826ec (-155, 1155)
        1155             155
010000111101 10 000011011110 001000 0x43d83788 (222, 1085)
        1085             222 ↳ sign of X coordinate?
010000111101 10 000011011110 001100 0x43d8378c (222, 1085)
        1085             222    ↳ set when warped away? (also set when leaving (-155, 1155))
111010000000 01 001000011000 001000 0xe8048608 (4632, -3712)
        3712 |           536
             | 1001000011000
             |          4632
             ↳ sign of Y coordinate?
```

That was helpful; it looks like the X coordinate is actually 13 bits plus a sign bit, and it looks like each of the coordinate fields is preceded by its sign. It's odd that the sign bits seem to have opposite meanings though, so that interpretation might be wrong.

In [ ]:
@dataclass(frozen=True)
class Footprint:
  x: int
  y: int
  warped_out: bool

  def __str__(self):
    flags = '' if self.warped_out else 'W'
    return f'({self.x:5},{self.y:5}){flags}'

  @classmethod
  def from_word(cls, value):
    y = (value & 0xfff00000) >> 20
    if (value & 0x00080000) == 0:
      y = -y

    x = (value & 0x0007ffc0) >> 6
    if (value & 0x00000020) != 0:
      x = -x

    warped_out = (value & 0x00000004) == 0

    return cls(x, y, warped_out)

for p in MORE_SAVES[1].points[-12:]:
  print(Footprint.from_word(p))

( -191, 1172)
( -178, 1168)
( -179, 1166)
( -165, 1158)
( -155, 1155)W
(  222, 1085)
(  222, 1085)W
( 4632,-3712)
( 4632,-3712)
( 4632,-3712)
( 4632,-3712)
( 4632,-3712)


This looks good! The next thing to consider is how we can tell what layer the player is on. Combine a few of the earlier samples with these, since the earlier ones were depths and these are surface.

```
001001010100 1 0000011111011 1 10000 0x25483ef0 (-249, 601) Depths
001010000011 1 0000011111010 1 10000 0x28383eb0 (-250, 633) Depths
010010000011 1 0000010011011 1 01100 0x483826ec (-155, 1155) Surface
010000111101 1 0000011011110 0 01000 0x43d83788 (222, 1085) Surface
010000111101 1 0000011011110 0 01100 0x43d8378c (222, 1085) Surface
111010000000 0 1001000011000 0 01000 0xe8048608 (4632, -3712) Surface
                                ↳ 2 bits, 1 = surface; 2 = depths
```

The BOTW documentation for trackblocks said there was one bit for `MainField` (overworld) or `AocField` (dungeon) points, and differentiating between dungeons or each of the three layers can be conveniently done in two bits, so it makes sense that we seem to have a two-bit field like this.

At this point we're missing only a few bit meanings:

 * Values 0 and 3 of the layer bits
 * Bit 0
 * Bit 1

Assuming the meanings of bits 0 and 1 are similar to some of the ones from BOTW, they might be:

 * Player auto-placed (after drowning but not dying?)
 * Player died
 * Riding a horse

# Some mapping

Since the goal is to get this displayed on an interactive map, let's pause to consider how to actually display a map. Fortunately other people have already done the work of extracting map images from the game:

 * https://www.zeldadungeon.net/tears-of-the-kingdom-interactive-map/
 * https://github.com/Slluxx/TOTK-Interactive-Map/

Both of these are open-source (though the Zelda Dungeon site doesn't seem to link to the sources; it's at https://github.com/zeldadungeon/maps).

In [30]:
from ipyleaflet import leaflet, LayersControl, Map

def zeldadungeon_layer(name):
  url = f'https://raw.githubusercontent.com/zeldadungeon/maps/develop/public/totk/tiles/{name}/{{z}}/{{x}}_{{y}}.jpg'
  return leaflet.TileLayer(
    url=url,
    max_zoom=6,
    tile_size=564,
    no_wrap=True,
    name=name.title(),
    base=True,
)

def totk_map():
  map = Map(
    layers=[
      zeldadungeon_layer('sky'),
      zeldadungeon_layer('depths'),
      zeldadungeon_layer('surface'),
    ],
    crs=leaflet.projections.Simple,
    zoom=2,
    center=(-564/2, 564/2)
  )

  map.add_control(LayersControl())
  return map

totk_map()

Map(center=[-282.0, 282.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

I had to experiment some to figure out the coordinate systems, but this wasn't too hard to get running. I'll probably have to experiment more to figure out the coordinate system transformation from game coordinates to this map, though. The [Leaflet documentation for `Simple` coordinate systems](https://leafletjs.com/examples/crs-simple/crs-simple.html) covers the basics that I'll need.

# The remaining data bits

Time to look at a few more saves to determine what the meaning of values 0 and 3 for the field in bits 3 and 4 of each footprint record, and see what the (presumably) flags in bits 0 and 1 do.

## Remaining layer values

I'm starting by assuming one value is for being in the sky, and the other in dungeons. I'm not sure what `AocField` is, but the [Zelda Mods wiki](https://zeldamods.org/wiki/Content/Map/AocField) says it's the area used for the Trial of the Sword. I don't think TOTK has a separate world like that, so it's possible the value 3 is unused.

To start, I'm guessing value 0 is the sky but let's test. I warped into the sky and walked around a bit before saving `sky.sav`.

In [ ]:
SKY_SAVE = Footprints.from_sav(MORE_SAVES_DIR / 'sky.sav', 358, -1658, Layer.SKY)

for word in SKY_SAVE.points[-16:]:
  print(f'{word:032b} {word:#010x}', Footprint.from_word(word))

01000011110110000011011110001000 0x43d83788 (  222, 1085)
01000011110110000011011110001100 0x43d8378c (  222, 1085)W
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001100 0xe804860c ( 4632,-3712)W
01100111010000000101110111000000 0x67405dc0 (  375,-1652)
01100111000100000101101011000000 0x67105ac0 (  363,-1649)
01100111001000000101011100000000 0x67205700 (  348,-1650)
01100111011000000101011100000000 0x67605700 (  348,-1654)
01100111101000000101011011000000 0x67a056c0 (  347,-1658)
01100111101000000101100101000000 0x67a05940 (  357,-1658)
01100111100000000101101010000000 0x67805a80 (  362,-1656)


These offsets don't look right.

In [ ]:
print(len(SKY_SAVE))
print(len(MORE_SAVES[1]))

51837
51835


This new save looks like it only has two more points saved, even though when I look at the path in-game it's showing new points where I'm running around in the sky. Try warping elsewhere that's not on the Great Sky Island.

In [ ]:
SKY_SAVE_2 = Footprints.from_sav(MORE_SAVES_DIR / 'sky2.sav', 2313, -1767, Layer.SKY)

for word in SKY_SAVE_2.points[-16:]:
  print(f'{word:032b} {word:#010x}', Footprint.from_word(word))

11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001000 0xe8048608 ( 4632,-3712)
11101000000001001000011000001100 0xe804860c ( 4632,-3712)W
01100111010000000101110111000000 0x67405dc0 (  375,-1652)
01100111000100000101101011000000 0x67105ac0 (  363,-1649)
01100111001000000101011100000000 0x67205700 (  348,-1650)
01100111011000000101011100000000 0x67605700 (  348,-1654)
01100111101000000101011011000000 0x67a056c0 (  347,-1658)
01100111101000000101100101000000 0x67a05940 (  357,-1658)
01100111100000000101101010000000 0x67805a80 (  362,-1656)
01100111011100000101110101000000 0x67705d40 (  373,-1655)
01100111011100000101110101000000 0x67705d40 (  373,-1655)
01100111011100000101110101000000 0x67705d40 (  373,-1655)
01100111001000000101111110000100 0x67205f84 (  382,-1650)W


Okay, so perhaps blocks get buffered and only written out later. Now we can see the steps I took over on the Great Sky Island, and as expected the layer bits are zero.

Time to see if we can identify any situations where the bottom-most two bits are set! The saves I'm loading have a lot of steps in them already, so see if they ever get set at all:

In [ ]:
for idx, word in enumerate(SKY_SAVE_2.points):
  if word & 3 != 0:
    print(f'{word:032b} {word:#010x}', idx, Footprint.from_word(word))

01001100010000000111010111000010 0x4c4075c2 47 (  471,-1220)
00110111111100000110011011000010 0x37f066c2 86 (  411, -895)
01011101111100000000011011000010 0x5df006c2 299 (   27,-1503)
01101011000000001000101100000010 0x6b008b02 661 (  556,-1712)
01010110101100001100000111000010 0x56b0c1c2 737 (  775,-1387)
01001010011000001011100000000010 0x4a60b802 918 (  736,-1190)
01001101111000001011101111000010 0x4de0bbc2 934 (  751,-1246)
01001110000000001011001100000010 0x4e00b302 956 (  716,-1248)
01001101110000001011110111000010 0x4dc0bdc2 982 (  759,-1244)
01101001100100000110101110000010 0x69906b82 1040 (  430,-1689)
01100011110000000110001001000010 0x63c06242 1069 (  393,-1596)
01100100111100000100100010000010 0x64f04882 1178 (  290,-1615)
01100001001000000111001000000010 0x61207202 1188 (  456,-1554)
00110000111110000101010110101010 0x30f855aa 1575 ( -342,  783)
00110001001110000101010011101010 0x313854ea 1579 ( -339,  787)
01000001011000001011001001101010 0x4160b26a 2598 ( -713,-1046)
011

Okay, so bit zero is never set in this save and bit 1 is rarely set. Bit 1 has been set on all three layers at various times as well, but a number of them appear out near (-4500, -3500) on the surface. That seems to be somewhere in the Gerudo Desert. In particular, they look like they're inside the Lightning Temple.

![Lightning Temple and surrounds, map](https://drive.google.com/uc?export=view&id=10ptWQ7nByN6qDkKCYrwHmWuScFH-n6JS)

We're not allowed to look at the Hero's Path when inside a dungeon; it instead toggles the dungeon map. However by leaving the dungeon and looking at the path, we can tell it's recorded normally while inside the dungeon. It also looks like there are many more points recorded than the ones we found with bit 1 set, so that's probably not a flag indicating we're simply inside a dungeon.

How about some of the other locations? In particular, the last death marker on the map is near (2350, -1782) in the sky. Is that close to one of the above points? Yes it is, and that's even the last one. I'm confident in saying that bit 1 is set to indicate a position that the player died in.

Have a peek at the time around that death to see if there's anything else interesting about it:



In [ ]:
for word in SKY_SAVE_2.points[27029 - 8:27029 + 8]:
  print(f'{word:032b} {word:#010x}', Footprint.from_word(word))

01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000010 0x6f624b82 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)
01101111011000100100101110000000 0x6f624b80 ( 2350,-1782)


It looks like it was time spent standing still, but that point is also the entrance to a shrine so these points probably represent time spent inside a shrine where the player died.

In [15]:
#@title Full-featured save representation
from dataclasses import dataclass
from enum import Enum

@dataclass(frozen=True)
class FootprintSav:
  footprints: tuple[int]

  @dataclass(frozen=True)
  class Footprint:
    """A single point of the Hero's Path."""
    class Layer(Enum):
      SKY = 0
      SURFACE = 1
      DEPTHS = 2
      UNKNOWN = 3

    layer: Layer
    x: int
    y: int
    warp: bool
    death: bool
    unknown_flag: bool

    _Y_COORD_SHIFT = 20
    _Y_COORD_MASK = ((1 << 12) - 1) << _Y_COORD_SHIFT
    _Y_COORD_POSITIVE_MASK = 1 << 19

    _X_COORD_SHIFT = 6
    _X_COORD_MASK = ((1 << 13) - 1) << _X_COORD_SHIFT
    _X_COORD_NEGATIVE_MASK = 1 << 5

    _LAYER_SHIFT = 3
    _LAYER_MASK = 0b11 << _LAYER_SHIFT

    _WARP_FLAG = 1 << 2
    _DEATH_FLAG = 1 << 1
    _UNKNOWN_FLAG = 1 << 0

    @classmethod
    def from_word(cls, value: int) -> 'Footprint':
      y = (value & cls._Y_COORD_MASK) >> cls._Y_COORD_SHIFT
      if (value & cls._Y_COORD_POSITIVE_MASK) == 0:
        y = -y

      x = (value & cls._X_COORD_MASK) >> cls._X_COORD_SHIFT
      if (value & cls._X_COORD_NEGATIVE_MASK) != 0:
        x = -x

      layer = cls.Layer((value & cls._LAYER_MASK) >> cls._LAYER_SHIFT)
      warp = (value & cls._WARP_FLAG) != 0
      death = (value & cls._DEATH_FLAG) != 0
      unknown_flag = (value & cls._UNKNOWN_FLAG) != 0

      return cls(layer, x, y, warp, death, unknown_flag)

    @staticmethod
    def _format_flag(value: bool, name: str) -> str:
      if value:
        return name
      else:
        return ' ' * len(name)

    def __str__(self):
      flags = self._format_flag(self.warp, 'W') + self._format_flag(self.death, 'D') + self._format_flag(self.unknown_flag, 'U')
      layer_tag = {
          self.Layer.SKY: 'Sky',
          self.Layer.SURFACE: 'Sur',
          self.Layer.DEPTHS: 'Dep',
      }.get(self.layer, 'Unk')
      return f'{layer_tag:3}({self.x:5},{self.y:5}){flags}'


  @classmethod
  def from_file(cls, path):
    with open(path, 'rb') as f:
      f.seek(76)
      count = int.from_bytes(f.read(4), byteorder='little')

      f.seek(364)
      points = []
      for _ in range(count):
        points.append(int.from_bytes(f.read(4), byteorder='little'))
      return cls(points)

  def __repr__(self):
    return f'<{type(self).__name__}; {len(self)} points>'

  def __len__(self):
    return len(self.footprints)

  def __getitem__(self, idx):
    if isinstance(idx, slice):
      return tuple(self.Footprint.from_word(w) for w in self.footprints[idx])
    return self.Footprint.from_word(self.footprints[idx])

  def __iter__(self):
    return (self.Footprint.from_word(p) for p in self.footprints)

MY_NICE_SAVE = FootprintSav.from_file(SAVES_DIR / 'slot_05' / 'footprint.sav')
MY_NICE_SAVE

<FootprintSav; 68240 points>

In [31]:
MY_NICE_DEATHS = tuple(p for p in MY_NICE_SAVE if p.death)
print('Found', len(MY_NICE_DEATHS), 'deaths')
for p in MY_NICE_DEATHS:
  print(p)

Found 67 deaths
Sky(  285,-1622) D 
Sky(  473,-1598) D 
Sky(  690,-1429) D 
Sky(  748,-1416) D 
Sky(  824,-1457) D 
Sky(  833,-1451) D 
Sky(  820,-1364) D 
Sur(  392,-1087) D 
Sur(  649,  873) D 
Sur(  651,  873) D 
Sur(  668,  881) D 
Sur(  691, 1261) D 
Sur(  546,  736) D 
Sur(  574,  779) D 
Sur(  547,  733) D 
Sur(  563,  779) D 
Sur(  561,  544) D 
Sur( -178,-1126) D 
Sur( -666,-1040) D 
Sur( -685,-1043) D 
Sur(-1950, -318) D 
Sur( -733,-1023) D 
Sur( -744,-1019) D 
Sur( -749,-1542) D 
Sur( -777,-1903) D 
Sur( -749,-1847) D 
Sur( -813,-1736) D 
Sur(-1295,-2197) D 
Sur(-1283,-2278) D 
Sur(-1274,-2267) D 
Sur(-1394,-1999) D 
Sur(-1174,-1952) D 
Sur(-1015,-2248) D 
Sur(-1119,-1288) D 
Sur(-1153,-1291) D 
Sur( 2768,-2262) D 
Sur( 4040,-1679) D 
Sur( 4434,  738) D 
Sur( 3856,  566) D 
Sur( 1541,  183) D 
Sur( 1556,  382) D 
Sur( 1551,  364) D 
Sur( 2467, 1637) D 
Sur( 2399, 1627) D 
Sur( 2645, 2762) D 
Sur( 4208, 3279) D 
Sur( 4184, 3295) D 
Sur( 3061, 1823) D 
Sur( 3061, 1823) D 
Sur(

# CRS transformation

Returning to mapping, the tiles we have project the world onto what ends up being 564 units wide, which is significantly narrower than game coordinates. Rather than do math and manually cross-reference with the game, we can look at the Zelda Dungeon map source code instead to find the correct transformation.

`src/totk.ts` sets up the Tears of the Kingdom map:

```typescript
  const options: ZDMapOptions = {
    directory: "totk",
    gameTitle: "Tears of the Kingdom",
    mapSizePixels: 36096,
    mapSizeCoords: 12032,
    tileSizePixels: 564,
    center: [101, -255],
  };

  const map = ZDMap.create(options);
```




## ZDMap

`ZDMap` is imported from elsewhere in the source tree and extends Leaflet's core `Map` class.

```typescript
  public static create(options: ZDMapOptions): ZDMap {
    const maxZoom = Math.round(
      Math.log(options.mapSizePixels / options.tileSizePixels) * Math.LOG2E
    );
    if (options.maxZoom == undefined) {
      options.maxZoom = maxZoom;
    }
    if (options.zoom == undefined) {
      options.zoom = maxZoom - 2;
    }

    const initZoom = Number(params.z);
    if (!isNaN(Number(params.z))) {
      options.zoom = initZoom;
    }
    let initLat = Number(params.x);
    if (isNaN(initLat)) {
      initLat = Number(params.lat);
    }
    let initLng = Number(params.y);
    if (isNaN(initLng)) {
      initLng = Number(params.lng);
    }
    if (!isNaN(initLat) && !isNaN(initLng)) {
      options.center = [initLat, initLng];
    }

    const crs = ZDCRS.create(
      options.mapSizeCoords ?? options.mapSizePixels,
      options.tileSizePixels
    );
    options.crs = crs;

    const bounds = new LatLngBounds(
      crs.pointToLatLng(new Point(0, options.mapSizePixels), maxZoom),
      crs.pointToLatLng(new Point(options.mapSizePixels, 0), maxZoom)
    );
    options.maxBounds = bounds.pad(0.5);

    options.zoomControl = false; // using a custom zoom control instead
    options.attributionControl = false;

    const map = new ZDMap(
      "map",
      options.directory,
      options.tileSizePixels,
      bounds,
      options
    );
```

It uses its own CRS (`ZDCRS`) that wraps Leaflet's core CRS class:

```typescript
import { CRS, Transformation, Util } from "leaflet";

export function create(mapSize: number, tileSize: number): L.CRS {
  const scale = tileSize / mapSize;
  const offset = tileSize / 2;
  const zdcrs = Util.extend({}, CRS.Simple, {
    transformation: new Transformation(scale, offset, -scale, offset),
  });

  return zdcrs;
}
```


`ipyleaflet` doesn't seem to expose enough of the javascript internals of Leaflet to allow us to create a custom CRS with transformation, so we'll prototype it in Python for now by reimplementing `Leaflet/src/geometry/Transformation.js`.

In [16]:
mapSize = mapSizeCoords = 12032
tileSize = tileSizePixels = 564

scale = tileSize / mapSize
offset = tileSize / 2

(scale, offset)

(0.046875, 282.0)

In [39]:
from collections import namedtuple

class Transformation(namedtuple('Transformation', ('a', 'b', 'c', 'd'))):
  def transform(self, x: float, y: float, scale=1.0) -> tuple[float, float]:
    return (
        scale * (self.c * y + self.d),
        scale * (self.a * x + self.b),
    )

MAP_TRANSFORM = Transformation(scale, offset, scale, -offset)
MAP_TRANSFORM

Transformation(a=0.046875, b=282.0, c=0.046875, d=-282.0)

In [36]:
from ipyleaflet import Marker

map = totk_map()

for p in MY_NICE_SAVE:
  if p.death:
    m = Marker(location=MAP_TRANSFORM.transform(p.x, p.y),
               title=f'{p} {MAP_TRANSFORM.transform(p.x, p.y)}', draggable=False)
    map.add(m)

map

Map(center=[-282.0, 282.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [50]:
from itertools import pairwise
from ipyleaflet import Polyline

map = totk_map()

chunks = []
chunk = []
for p1, p2 in pairwise(MY_NICE_SAVE):
  if p1.warp or p1.death:
    chunks.append(chunk)
    chunk = []
    continue

  chunk.append(MAP_TRANSFORM.transform(p1.x, p1.y))
  chunk.append(MAP_TRANSFORM.transform(p2.x, p2.y))

if chunk:
  chunks.append(chunk)

map.add(Polyline(locations=chunks, name="Hero's Path",
                 color='rgba(0, 255, 255, 0.5)', weight=3, fill=False))
map

Map(center=[-282.0, 282.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…